In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

try:
    import scikeras
except ImportError:
    !python -m pip install scikeras

from scikeras.wrappers import KerasClassifier, KerasRegressor

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
    
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão. 

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
# Baixa a base de dados.
housing = fetch_california_housing()

# Divide o conjunto total de exemplos em conjuntos de treinamento e teste.
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Divide o conjunto de treinamento em conjuntos de treinamento (menor) e validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Aplica padronização às matrizes de atributos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Criando o modelo a ser otimizado

Para realizar a otimização com as classes `GridSearchCV` ou `RandomizedSearchCV` da biblioteca SciKit-Learn, **precisamos encapsular nossos modelos em objetos que emulem regressores (ou classificadores) da biblioteca Scikit-Learn**. 


O primeiro passo é **criar uma função que irá construir e compilar um modelo do Tensorflow**.


Esta função cria um modelo com a API `Sequencial` para regressão, com a dimensão da entrada e o número de camadas ocultas e neurônios, e o compila usando um otimizador `SGD` configurado com a taxa de aprendizado fornecida.

In [4]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    # Instantiate a Sequential object.
    model = keras.models.Sequential()
    
    # Declare the model
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    
    # Define the optimizer and complile the model.
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

### Regressor Keras

Em seguida, criamos um objeto da classe `KerasRegressor` baseado na função `build_model()`.


O objeto da classe `KerasRegressor` é um **wrapper** (i.e., embrulho) em torno do modelo construído usando a função `build_model()`.


**OBS**.: Passamos para o construtor da classe `KerasRegressor` a função e os parâmetros, definidos na função, que iremos alterar durante o ajuste dos hiperparâmetros.

In [5]:
keras_reg = KerasRegressor(build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)

Agora podemos usar esse objeto como um regressor do Scikit-Learn: 
+ podemos treiná-lo usando seu método `fit()`, 
+ avaliá-lo usando seu método `score()` 
+ e usá-lo para fazer predições usando seu método `predict()`.

In [6]:
keras_reg.fit(X_train, y_train, 
              epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)]
             )

Epoch 1/100
363/363 [==============================] - 2s 3ms/step - loss: 1.0896 - val_loss: 20.7721
Epoch 2/100
363/363 [==============================] - 1s 3ms/step - loss: 0.7606 - val_loss: 5.0266
Epoch 3/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5456 - val_loss: 0.5490
Epoch 4/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4732 - val_loss: 0.4529
Epoch 5/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4503 - val_loss: 0.4188
Epoch 6/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4338 - val_loss: 0.4129
Epoch 7/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4241 - val_loss: 0.4004
Epoch 8/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4168 - val_loss: 0.3944
Epoch 9/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4108 - val_loss: 0.3961
Epoch 10/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4060 - val_l

KerasRegressor(
	model=<function build_model at 0x0000029EAE917550>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	n_hidden=1
	n_neurons=30
	learning_rate=0.003
)

In [7]:
mse_test = keras_reg.score(X_test, y_test)

print('MSE test:', mse_test)

162/162 [==============================] - 0s 2ms/step
MSE test: 0.7474111269910106


In [8]:
X_new = X_test[:3]

y_pred = keras_reg.predict(X_new)

for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i]))

1/1 [==============================] - 0s 30ms/step
Predictions: [0.6034199 1.5396053 4.1040044]


### Otimização hiperparamétrica
 
Agora vamos treinar vários modelos com diferentes valores para os hiperparâmetros e ver qual conjunto de valores tem o melhor desempenho no conjunto de validação. 


Como existem muitos hiperparâmetros a serem testados, é preferível usar uma pesquisa aleatória (`RandomSearchCV`) em vez de uma pesquisa em grade (`GridSearch`).


Vamos explorar o **número de camadas ocultas**, o **número de neurônios** e o **passo de aprendizado**.

**OBS**.:

+ Listas de valores passados para os hiperparâmetros são amostradas uniformemente.
+ Ao invés de listas, pode-se passar distribuições, de onde valores são amostrados (e.g., a distribuição `reciprocal`).
+ O parâmetro `n_iter` define o número de configurações (i.e., combinações) de hiperparâmetros que são amostradas.

In [9]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, scoring='neg_mean_squared_error', cv=3, verbose=2)

rnd_search_cv.fit(X_train, y_train, 
                  epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)]
                 )

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
242/242 [==============================] - 1s 4ms/step - loss: 3.3071 - val_loss: 1.3735
Epoch 2/100
242/242 [==============================] - 1s 3ms/step - loss: 0.9782 - val_loss: 0.6881
Epoch 3/100
242/242 [==============================] - 1s 3ms/step - loss: 0.6186 - val_loss: 0.5887
Epoch 4/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5504 - val_loss: 0.5466
Epoch 5/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5363 - val_loss: 0.5360
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5307 - val_loss: 0.7261
Epoch 7/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5322 - val_loss: 0.6728
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5311 - val_loss: 0.6939
Epoch 9/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5326 - val_loss: 0.5351
Epoch 10/100
242/242 [========

Epoch 81/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5246 - val_loss: 0.8111
Epoch 82/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5307 - val_loss: 0.5353
Epoch 83/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5280 - val_loss: 0.4954
Epoch 84/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5272 - val_loss: 0.5445
Epoch 85/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5270 - val_loss: 0.6084
Epoch 86/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5272 - val_loss: 0.6462
Epoch 87/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5290 - val_loss: 0.4923
Epoch 88/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5276 - val_loss: 0.5009
Epoch 89/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5257 - val_loss: 0.6698
Epoch 90/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5275

242/242 [==============================] - 0s 2ms/step - loss: 0.5332 - val_loss: 0.4935
Epoch 38/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5298 - val_loss: 0.7451
Epoch 39/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5344 - val_loss: 0.5824
Epoch 40/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5319 - val_loss: 0.6109
Epoch 41/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5295 - val_loss: 0.8136
Epoch 42/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5340 - val_loss: 0.6256
Epoch 43/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5316 - val_loss: 0.5192
Epoch 44/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5300 - val_loss: 0.7006
Epoch 45/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5302 - val_loss: 0.7742
Epoch 46/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5315 - val_loss: 

242/242 [==============================] - 0s 2ms/step - loss: 19965354.0000 - val_loss: 5623153152.0000
Epoch 16/100
242/242 [==============================] - 0s 2ms/step - loss: 100124888.0000 - val_loss: 22019987456.0000
Epoch 17/100
242/242 [==============================] - 0s 2ms/step - loss: 490005120.0000 - val_loss: 86582329344.0000
Epoch 18/100
242/242 [==============================] - 0s 2ms/step - loss: 2112389888.0000 - val_loss: 340066598912.0000
Epoch 19/100
242/242 [==============================] - 1s 2ms/step - loss: 8082988544.0000 - val_loss: 1331716816896.0000
Epoch 20/100
242/242 [==============================] - 0s 2ms/step - loss: 31350368256.0000 - val_loss: 5207425548288.0000
Epoch 21/100
242/242 [==============================] - 0s 2ms/step - loss: 58745643008.0000 - val_loss: 20485045747712.0000
Epoch 22/100
242/242 [==============================] - 0s 2ms/step - loss: 456048214016.0000 - val_loss: 80743405453312.0000
Epoch 23/100
242/242 [=============

Epoch 78/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 79/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 80/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 81/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 82/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 83/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 84/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 85/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 86/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 87/100
242/242 [==============================] - 0s 2ms/step - loss: inf - val_loss: inf
Epoch 88/100
242/242 [==================

...

242/242 [==============================] - 1s 4ms/step - loss: 0.3200 - val_loss: 0.3851
Epoch 57/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3171 - val_loss: 0.5022
Epoch 58/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3194 - val_loss: 0.3460
Epoch 59/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3153 - val_loss: 0.3389
Epoch 60/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3150 - val_loss: 0.3372
Epoch 61/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3147 - val_loss: 0.4234
Epoch 62/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3143 - val_loss: 0.3097
Epoch 63/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3125 - val_loss: 0.3548
Epoch 64/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3125 - val_loss: 0.3192
Epoch 65/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3107 - val_loss: 

242/242 [==============================] - 1s 3ms/step - loss: 0.3359 - val_loss: 0.5610
Epoch 36/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3346 - val_loss: 0.5112
Epoch 37/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3342 - val_loss: 0.5005
Epoch 38/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3333 - val_loss: 0.5595
Epoch 39/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3319 - val_loss: 0.4900
Epoch 40/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3313 - val_loss: 0.5240
Epoch 41/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3298 - val_loss: 0.5280
Epoch 42/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3295 - val_loss: 0.5336
Epoch 43/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3285 - val_loss: 0.4530
Epoch 44/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3276 - val_loss: 

242/242 [==============================] - 1s 3ms/step - loss: 0.3794 - val_loss: 0.4330
Epoch 15/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3765 - val_loss: 0.4049
Epoch 16/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3734 - val_loss: 0.4303
Epoch 17/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3709 - val_loss: 0.4014
Epoch 18/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3689 - val_loss: 0.3938
Epoch 19/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3668 - val_loss: 0.4229
Epoch 20/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3649 - val_loss: 0.4164
Epoch 21/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3640 - val_loss: 0.4102
Epoch 22/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3619 - val_loss: 0.4027
Epoch 23/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3601 - val_loss: 

363/363 [==============================] - 1s 3ms/step - loss: 0.3119 - val_loss: 0.4377
Epoch 74/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3095 - val_loss: 0.3067
Epoch 75/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3073 - val_loss: 0.3736
Epoch 76/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3082 - val_loss: 0.3597
Epoch 77/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3085 - val_loss: 0.3852
Epoch 78/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3068 - val_loss: 0.3075
Epoch 79/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3077 - val_loss: 0.3369
Epoch 80/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3077 - val_loss: 0.3023
Epoch 81/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3059 - val_loss: 0.3115
Epoch 82/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3033 - val_loss: 

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(learning_rate=0.003, model=<function build_model at 0x0000029EAE917550>, n_hidden=1, n_neurons=30),
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029EB2583040>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   verbose=2)

A busca pode durar muitas horas dependendo do hardware, do tamanho do conjunto de dados, da complexidade do modelo e do valor de `n_iter` e `cv`. 

Quando terminar, podemos acessar

+ os melhores hiperparâmetros encontrados, 
+ a melhor pontuação 
+ e o melhor modelo treinado

In [10]:
# Melhores hiperparâmetros.
rnd_search_cv.best_params_

{'learning_rate': 0.0033625641252688094, 'n_hidden': 2, 'n_neurons': 42}

In [11]:
# Score do melhor conjunto de hiperparâmetros.
rnd_search_cv.best_score_

0.7591441682575105

In [12]:
# Melhor modelo.
model = rnd_search_cv.best_estimator_.model

#### Conclusões

+ A pesquisa aleatória funciona bem (encontra o melhor conjunto de hiperparâmetros) para muitos problemas simples. 


+ No entanto, quando temos problemas mais complexos, com conjuntos de dados maiores, essa abordagem explorará apenas uma pequena parte do espaço de hiperparâmetros e pode não encontrar o melhor conjunto de valores. 


+ Podemos aliviar parcialmente esse problema auxiliando o processo de pesquisa manualmente: 
    + Primeiro executamos uma **pesquisa aleatória rápida usando grandes intervalos de valores de hiperparâmetros**, 
    + Na sequência, executamos outra pesquisa usando **intervalos menores, centrados nos melhores valores encontrados** durante a primeira execução e assim por diante. 


+ Esperamos que isso ajude a encontrar um bom conjunto de hiperparâmetros. No entanto, isso consome muito tempo.


+ Felizmente, existem outras formas mais eficientes (i.e., inteligentes) de se realizar a otimização dos hiperparâmetros.